In [1]:
%pylab inline
import pandas as pd
import numpy as np

import torch
import matplotlib.pyplot as plt

import os
from tqdm import tqdm                                # for progress bar
from sklearn.model_selection import train_test_split
from sklearn.cluster import AgglomerativeClustering, KMeans, SpectralClustering
from sklearn.metrics import adjusted_rand_score
from sklearn import preprocessing

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("running on the gpu")
else:
    device = torch.device("cpu")
    print("running on the cpu")
    
from compression import pipeline

Populating the interactive namespace from numpy and matplotlib
running on the gpu


In [2]:
path = "goolam"
do_clus = True
latent = pipeline(path_name=path, is_plot_denoise=False, vae_choice='paper', retrain=True) # a list with 3 differently trained latent variables
print(len(latent), latent[0].shape)


log2 is applied
first column of normalized_data
[0.45673727 0.41277488 0.45339973 0.34339266 0.39761435 0.47851649
 0.42687017 0.41511023 0.33093438 0.35535198 0.36670658 0.45023024
 0.40043435 0.45196978 0.37736807 0.39095186 0.23418182 0.39985241
 0.46387343 0.4330633  0.37242285 0.3996168  0.34810324 0.45229824
 0.35513868 0.45978068 0.44874997 0.47019561 0.45432799 0.50368729
 0.43392305 0.40690908 0.36245137 0.42567683 0.42778335 0.441645
 0.49527818 0.47426508 0.45409    0.43216159 0.45311978 0.45456565
 0.51180229 0.46854753 0.37801226 0.42012368 0.4831555  0.45191327
 0.42939831 0.48954521 0.48692279 0.46442425 0.44307403 0.36137441
 0.49888944 0.41050206 0.39993837 0.39971253 0.38087504 0.47462718
 0.3897801  0.42853193 0.29358917 0.50614974 0.49537921 0.49197499
 0.44618553 0.45329972 0.46797146 0.47187686 0.4864775  0.49805648
 0.43143022 0.2958528  0.20408614 0.31025091 0.3421828  0.34659626
 0.3576697  0.30695883 0.35534803 0.33608001 0.09025048 0.31171175
 0.35811395 0.35

100%|██████████| 62/62 [00:00<00:00, 283.86it/s]


epoch 9 Loss:  0.19321280717849731
paper_encoder(
  (encoder): Linear(in_features=124, out_features=64, bias=True)
  (bn): BatchNorm1d(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
  (mu): Linear(in_features=64, out_features=15, bias=True)
  (var): Linear(in_features=64, out_features=15, bias=True)
  (sample_expander): ModuleList(
    (0): Linear(in_features=15, out_features=64, bias=True)
    (1): Linear(in_features=15, out_features=64, bias=True)
  )
  (decoder): ModuleList(
    (0): Linear(in_features=64, out_features=124, bias=True)
    (1): Linear(in_features=64, out_features=124, bias=True)
  )
)
training stacked bayesian autoencoder

###############################
#phase 1: the warm-up process#
######################################


100%|██████████| 62/62 [00:00<00:00, 133.40it/s]


epoch 9 Loss: 0.03833863139152527

##############################
#phase 2: the VAE stage#
############################


100%|██████████| 62/62 [00:00<00:00, 128.04it/s]


epoch 9 Loss: 1.8543415069580078


100%|██████████| 62/62 [00:00<00:00, 101.41it/s]

epoch 19 Loss: 1.8637399673461914
[[ 0.41278243  0.14169373 -0.46996102 ...  0.05423163  0.04568342
   0.00834418]
 [ 0.4416948   0.22213012 -0.4397324  ...  0.02734448 -0.02954184
   0.11566014]
 [ 0.02712245  0.01052719 -0.08650968 ...  0.00842969  0.01560827
   0.01188319]
 ...
 [ 0.02712245  0.01052719 -0.08650968 ...  0.00842969  0.01560827
   0.01188319]
 [-0.17385882  0.00830169 -0.6941473  ... -0.5200063   0.21937646
   0.31812555]
 [ 0.30791172 -0.18086055 -0.12965988 ... -0.5453196   0.18314217
   0.45523423]] 1.3928463 -1.3364081
124 (15,)


In [7]:
clustering = SpectralClustering(5, eigen_solver='arpack', affinity='nearest_neighbors', n_neighbors=7,  
                                                        assign_labels='kmeans', n_jobs=-1, random_state=0)  # n_jobs = -1 means use all processors    
clustering.fit(X_train)
print(clustering.labels_, y_train)
score = adjusted_rand_score(clustering.labels_, y_train)
print(score)

[1 3 4 4 4 2 3 3 0 4 4 3 2 3 4 1 1 2 1 2 4 4 4 2 2 3 0 2 4 2 4 4 3 4 3 4 1
 1 4 0 4 2 4 3 0 4 2 4 3 1 3 3 4 3 3 4 4 4 3 4 1 4 3 1 1 0 4] [0 3 4 4 2 4 3 3 2 5 1 0 4 0 5 0 0 4 3 4 4 1 4 4 4 3 2 4 0 4 0 4 0 4 3 4 3
 3 4 2 2 3 2 0 2 1 3 4 3 4 0 3 4 4 0 5 4 1 3 1 0 3 3 0 3 3 4]
0.16990115697659067


## Classification

In [4]:
path_name = 'goolam'

X = np.load(f'latent/{path_name}.npy')
# print(X)
y_df = pd.read_csv(f'rds_csv_data/{path_name}_labels.csv')['x']
y = np.array(y_df)
# le = preprocessing.LabelEncoder()
# y = le.fit_transform(y)


X_train, X_test, y_train, y_test = train_test_split(X, y)

from sklearn.neighbors import KNeighborsClassifier

from scipy.spatial.distance import correlation
neigh = KNeighborsClassifier(n_neighbors=5, metric=correlation, n_jobs=-1)
neigh.fit(X_train, y_train)
pred_y = neigh.predict(X_test)
print(y_test)
print(pred_y)
score = adjusted_rand_score(pred_y, y_test)
print(score)


['4cell' 'blast' '4cell' '4cell' '8cell' '4cell' '4cell' '8cell' '4cell'
 '8cell' '4cell' '8cell' '4cell' '2cell' '4cell' '2cell' '4cell' '4cell'
 '8cell' '8cell' '8cell' '8cell' 'blast' '8cell' '4cell' '4cell' '4cell'
 '4cell' '4cell' '4cell' '4cell']
['2cell' '4cell' '4cell' '4cell' '4cell' '4cell' '4cell' '4cell' '4cell'
 '8cell' '4cell' '8cell' '4cell' '2cell' '4cell' '4cell' '2cell' '4cell'
 '4cell' '4cell' '4cell' '16cell' '4cell' '8cell' '2cell' '4cell' '4cell'
 '4cell' '4cell' '4cell' '4cell']
0.12098298676748583


## Dimension reduction and visualization

In [ ]:
def scDHA_w(data):
    K = 3
    sample_prob = None
    do_clus = True
    gen_fil = True
    k = None
    # if sparse:
    #     # don't understand original R code
    #     '''
    #     if(sparse){
    #         if (min(data) < 0) stop("The input must be a non negative sparse matrix")
    #         if (max(data) - min(data) > 100){
    #             data@x <- log2(data@x + 1) what is data@x ??
    #         }
    #         data <- normalize_data_sparse(data)
    #     }
    #     ''' 
    #     if data.min() < 0:
    #         exit("The input must be a non negative sparse matrix")
           
    #     if data.max() - data.min() > 100:
    #         pass
    # else:
    #     if data.max() - data.min() > 100:
    #         if data.min() < 0:
    #             data -= data.min(axis=0)
    #         data = np.log2(data + 1)
        
    

## The following code is skipped (wMetaC part)

In [ ]:

if do_clus:
    # Use an ensemble of data projection models to achieve higher accuracy and to avoid local minima, not needed if we use kmeans++
    # first repeat the data projection
    labels = []
    for hidden in latent:
        # labels.append(clus(hidden, k=6, nmax=100))
        labels.append(clus(hidden, nmax=10))
    labels = np.array(labels) 
    print(labels)   
    S = np.zeros((len(labels), len(labels)))  # chance that cell i and j are in the same cluster
    for i, row in enumerate(S):
        for j, _ in enumerate(S):
            if not (i==j):
                S[i, j] = adjusted_rand_score(labels[i], labels[j])
    for i, row in enumerate(S):
        S[i,i] = row.mean()
    print(S)
    found = False
    if (S[S < 0.7]).sum() > 0:
        i = 2
    else:
        i = 1
        
    # find best guessed label (latent variable)
    while not found:
        # print(f'i={i}')
        tmp = KMeans(n_clusters = i, n_init = 100, max_iter = 5000).fit(S)
        k = tmp.labels_
        max = 0
        for c in range(tmp.cluster_centers_.shape[0]): # for k clusters
            score = S[k == c, k == c].mean()
            if score > max and (k==c).sum() > 1:
                max = score
                idx = (k == c)
        if max > 0.8:
            found = True
        if i >= 3:
            found = True
        
        i += 1
    # guess number of clusters
    tmp = []
    for label in labels[idx]:
        tmp.append(np.unique(label).shape[0])
        print(tmp)
    cluster_max = np.floor(np.mean(tmp)+0.5).astype(np.int)
    
    
    # (i) calculate cell-cell weighted similarity matrix 
    W = S * (1 - S)
    print(W.max(), W.min())
    # then combine the clustering results using the wMetaC
    # wMetaC = AgglomerativeClustering(n_clusters=k_classes, linkage='ward')
    # # wMetaC = AgglomerativeClustering(n_clusters=k_classes, affinity='precomputed')
    # wMetaC.fit(latent)
    # # wMetaC.fit(clustered.affinity_matrix_.toarray())
    # print(wMetaC)
    # print(clustered.labels_)
    # print(wMetaC.labels_)

        # print(latent.size())
    
    


In [ ]:
pipeline('campbell', 1024, 10, False, 1024, [10, 20], vae_choice='paper', retrain=True)